<a href="https://colab.research.google.com/github/Lamiaka/GNN_amld2022/blob/main/Starter_%7C_AMLD_2021_%7C_GNNs_for_Structured_Data_Workshop_by_Volodya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README

## Starter code for train any model warm up exercise

This is an example colab for Graph Neural Networks for Strucutred Data Workshop @ AMLD 2021 by volodya@google.com

Link to this colab: https://bit.ly/gnn-str-start



# Run this section now

It installs the libs

In [ ]:
!pip install spektral

!wget 'https://drive.google.com/uc?id=1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK&export=download' -O stocks-with-label_3days.prq

     |████████████████████████████████| 123 kB 8.8 MB/s 
--2021-09-28 00:37:25--  https://drive.google.com/uc?id=1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK&export=download
Resolving drive.google.com (drive.google.com)... 74.125.195.113, 74.125.195.101, 74.125.195.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ifistt5r00bh1rb2dm4klc2kj06f5nhk/1632789375000/01333786639642341364/*/1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK?e=download [following]
--2021-09-28 00:37:27--  https://doc-08-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ifistt5r00bh1rb2dm4klc2kj06f5nhk/1632789375000/01333786639642341364/*/1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK?e=download
Resolving doc-08-5c-docs.googleusercontent.com (doc-08-5c-docs.googleusercontent.com)... 108.177.98.132, 2607:f8b0:400e:c06::84
C

# Common imports and functionality

In [ ]:
import tensorflow as tf
from datetime import date
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
stocks_df = pd.read_parquet('stocks-with-label_3days.prq')

stocks_df.sample(5)

,ticker,Date,LABEL,High,Low,Close,Open,High_1,Low_1,Close_1,Open_1
144504,DIS,1979-12-27,1.01,0.62,0.61,0.61,0.61,0.61,0.58,0.61,0.59
125466,BRK-B,2018-09-14,1.00,216.51,215.30,216.37,215.50,215.42,214.17,214.89,215.02
116435,BAC,2008-01-15,0.95,30.96,30.17,30.23,30.96,31.36,30.65,31.30,30.93
175580,AAPL,2014-06-05,1.01,20.86,20.65,20.80,20.76,20.82,20.44,20.72,20.48
77250,WMT,2018-03-12,0.99,84.05,82.58,82.76,83.35,83.37,82.25,83.37,82.92


In [ ]:
print('# Examples', len(stocks_df))
stocks_df.describe().transpose().drop(columns=['count'])

# Examples 177164


,mean,std,min,25%,50%,75%,max
LABEL,1.00,0.03,0.49,0.99,1.00,1.02,1.68
High,68.05,216.16,0.01,3.30,19.59,50.78,"3,773.08"
Low,66.60,211.48,0.01,3.20,19.09,49.86,"3,696.79"
Close,67.35,213.88,0.01,3.25,19.34,50.33,"3,731.41"
Open,67.30,213.91,0.00,3.21,19.34,50.32,"3,744.00"
High_1,68.02,215.96,0.01,3.31,19.60,50.79,"3,773.08"
Low_1,66.58,211.28,0.01,3.21,19.10,49.87,"3,696.79"
Close_1,67.33,213.67,0.01,3.26,19.36,50.35,"3,731.41"
Open_1,67.28,213.71,0.00,3.23,19.36,50.33,"3,744.00"


In [ ]:
ticker_column = pd.Categorical(stocks_df['ticker'])
stocks_df['ticker'] = ticker_column
stocks_df['ticker'] = stocks_df.ticker.cat.codes

In [ ]:
split_config = {
    'test_dt': pd.Timestamp('2020-01-01'), # 2020 onwards goes into test.
    'valid_dt': pd.Timestamp('2015-01-01'),
}

test_df = stocks_df.query('Date >= @test_dt', local_dict=split_config).copy()
train_df = stocks_df.query('Date < @valid_dt', local_dict=split_config).copy()
valid_df = stocks_df.query('Date >= @valid_dt and Date < @test_dt', local_dict=split_config).copy()

print(f'Train: {len(train_df)} Valid: {len(valid_df)} Test: {len(test_df)}')

Train: 139260 Valid: 28934 Test: 8970
